### 실행 순서

이미지 키워드 만들기

azure 업로드

### api 키 불러오기 확인용

In [1]:

# main.py 또는 api_service.py 등
import os
from dotenv import load_dotenv
import openai

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# os.environ을 사용하여 환경 변수를 가져옵니다.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
STABILITY_API_KEY = os.getenv("STABILITY_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되어 있지 않습니다.")

print(OPENAI_API_KEY[:10])
print(STABILITY_API_KEY[:10])

# 이제 'client' 객체를 사용하여 API를 호출할 수 있습니다.

sk-svcacct
sk-tehCJ4Y


### 단일 이미지 만들기 (txt to img, img to img)

In [6]:
import os
import requests
import base64
from dotenv import load_dotenv
import openai
from PIL import Image
import io

# .env 파일에서 환경 변수 로드
load_dotenv()

# --- 핵심 함수들 (이전과 거의 동일) ---

def translate_with_gpt(text_to_translate: str) -> str:
    """
    OpenAI의 GPT 모델을 사용하여 텍스트를 영어로 번역합니다.
    이미지 생성에 적합하도록 구체적이고 묘사적인 문구로 변환합니다.
    """
    try:
        client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

        if not client.api_key:
            raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")

        print("🔄 프롬프트를 영어로 번역하고 다듬는 중...")
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an AI assistant that translates a Korean noun phrase into a specific, descriptive English phrase suitable for an image generation AI. Make it photorealistic and detailed."},
                {"role": "user", "content": f"""
                    Translate the following Korean phrase into a descriptive English phrase.
                    Korean: 소파에 앉아 편안하게 책 읽는 모습
                    English: A person sitting comfortably on a sofa, deeply engrossed in reading a book in a cozy living room.
                    Korean: {text_to_translate}
                    English:
                 """}
            ],
            temperature=0.2,
            max_tokens=80
        )
        
        translated_text = response.choices[0].message.content.strip()
        print(f"   - 번역 결과: {translated_text}")
        return translated_text if translated_text else ""

    except Exception as e:
        raise Exception(f"OpenAI API 오류: {e}")


def generate_final_image(english_prompt: str, output_filename: str):
    """
    텍스트를 기반으로 이미지를 생성하고, 지정된 경로에 저장합니다.
    """
    # ... (내부 로직은 이전과 동일) ...
    style_keywords = [
        'style', 'anime', 'cartoon', 'painting', 'drawing', 'sketch',
        'gogh', 'picasso', 'dali', 'monet', 'watercolor'
    ]
    has_style_request = any(keyword in english_prompt.lower() for keyword in style_keywords)
    
    final_prompt = ""
    style_preset = "photographic"

    if has_style_request:
        print("🎨 스타일 요청 감지.")
        final_prompt = english_prompt
        if 'anime' in english_prompt.lower(): style_preset = 'anime'
        elif 'comic' in english_prompt.lower(): style_preset = 'comic-book'
        else: style_preset = 'digital-art'
    else:
        print("📸 'photographic' 스타일로 이미지를 생성합니다.")
        photorealistic_prefix = "An ultra-realistic RAW photo of "
        photorealistic_suffix = ". Shot on a Sony α7 IV with an 85mm f/1.4 lens, 8K, cinematic lighting, soft shadows."
        final_prompt = photorealistic_prefix + english_prompt + photorealistic_suffix

    print(f"   - 최종 프롬프트: {final_prompt}")
    
    api_host = 'https://api.stability.ai'
    api_key = os.getenv("STABILITY_API_KEY")
    engine_id = "stable-diffusion-xl-1024-v1-0"

    if not api_key:
        raise ValueError("STABILITY_API_KEY가 .env 파일에 설정되지 않았습니다.")
        
    print("   - 이미지 생성 요청 중...")
    
    response = requests.post(
        f"{api_host}/v1/generation/{engine_id}/text-to-image",
        headers={"Content-Type": "application/json", "Accept": "application/json", "Authorization": f"Bearer {api_key}"},
        json={
            "text_prompts": [{"text": final_prompt}], "cfg_scale": 7, "height": 1024,
            "width": 1024, "samples": 1, "steps": 30, "style_preset": style_preset
        },
    )

    if response.status_code != 200:
        raise Exception(f"Stability AI API 오류: {response.status_code} - {response.text}")

    data = response.json()
    
    image_bytes = base64.b64decode(data["artifacts"][0]["base64"])
    img = Image.open(io.BytesIO(image_bytes))
    
    print(f"\n 생성된 이미지를 512x512로 리사이즈 중...")
    img_resized = img.resize((512, 512), Image.Resampling.LANCZOS)
    img_resized.save(output_filename, "jpeg")
    
    print(f"✅ 이미지가 '{output_filename}' 이름으로 저장되었습니다.")


# --- 메인 실행 로직 (단일 입력 방식으로 변경) ---

if __name__ == "__main__":
    try:
        # 1. 사용자로부터 직접 한글 프롬프트 입력받기
        korean_prompt = input("🎨 생성할 이미지에 대해 한글로 설명해주세요: ")

        # 2. 프롬프트 번역
        english_prompt = translate_with_gpt(korean_prompt)
        
        # 3. 이미지 생성 (고정된 파일명 사용)
        output_file = "generated_image.jpeg"
        generate_final_image(english_prompt, output_filename=output_file)
        
        print("\n🎉 작업 완료!")

    except Exception as e:
        print(f"\n--- ⚠️ 오류 발생 ---")
        print(e)
        print("--------------------")

🔄 프롬프트를 영어로 번역하고 다듬는 중...
   - 번역 결과: A person sitting comfortably on a sofa, deeply engrossed in reading a book in a cozy living room.
📸 'photographic' 스타일로 이미지를 생성합니다.
   - 최종 프롬프트: An ultra-realistic RAW photo of A person sitting comfortably on a sofa, deeply engrossed in reading a book in a cozy living room.. Shot on a Sony α7 IV with an 85mm f/1.4 lens, 8K, cinematic lighting, soft shadows.
   - 이미지 생성 요청 중...

 생성된 이미지를 512x512로 리사이즈 중...
✅ 이미지가 'generated_image.jpeg' 이름으로 저장되었습니다.

🎉 작업 완료!


### rag 이미지 데이터 한번에 만들기
- 필요한걸 csv로 목록화해서 넘기기

In [4]:
import os
import requests
import base64
import json
from dotenv import load_dotenv
import openai
from PIL import Image
import io
import pandas as pd # CSV 처리를 위해 추가

# .env 파일에서 환경 변수 로드
load_dotenv()

def translate_with_gpt(text_to_translate: str) -> str:
    """
    OpenAI의 GPT 모델을 사용하여 텍스트를 영어로 번역합니다.
    이미지 생성에 적합하도록 구체적이고 묘사적인 문구로 변환합니다.
    """
    try:
        client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

        if not client.api_key:
            raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an AI assistant that translates a Korean noun phrase into a specific, descriptive English phrase suitable for an image generation AI. Make it photorealistic and detailed."},
                {"role": "user", "content": f"""
                    Translate the following Korean phrase into a descriptive English phrase.
                    Korean: 소파에 앉아 편안하게 책 읽는 모습
                    English: A person sitting comfortably on a sofa, deeply engrossed in reading a book in a cozy living room.
                    Korean: {text_to_translate}
                    English:
                 """}
            ],
            temperature=0.2,
            max_tokens=80
        )
        
        translated_text = response.choices[0].message.content.strip()
        return translated_text if translated_text else ""

    except Exception as e:
        raise Exception(f"OpenAI API 오류: {e}")


def generate_final_image(english_prompt: str, output_filename: str, source_image_path: str = None):
    """
    텍스트나 이미지를 기반으로 이미지를 생성하고,
    결과물을 512x512로 리사이즈하여 지정된 경로에 저장합니다.
    """
    style_keywords = [
        'style', 'anime', 'cartoon', 'painting', 'drawing', 'sketch',
        'gogh', 'picasso', 'dali', 'monet', 'watercolor'
    ]
    has_style_request = any(keyword in english_prompt.lower() for keyword in style_keywords)
    
    final_prompt = ""
    style_preset = "photographic" # 기본 스타일을 'photographic'으로 설정

    if has_style_request:
        print("🎨 스타일 요청 감지.")
        final_prompt = english_prompt
        if 'anime' in english_prompt.lower(): style_preset = 'anime'
        elif 'comic' in english_prompt.lower(): style_preset = 'comic-book'
        else: style_preset = 'digital-art'
    else:
        print("📸 'photographic' 스타일로 이미지를 생성합니다.")
        photorealistic_prefix = "An ultra-realistic RAW photo of "
        photorealistic_suffix = ". Shot on a Sony α7 IV with an 85mm f/1.4 lens, 8K, cinematic lighting, soft shadows."
        final_prompt = photorealistic_prefix + english_prompt + photorealistic_suffix

    print(f"   - 최종 프롬프트: {final_prompt}")
    print(f"   - 스타일 프리셋: {style_preset}")
    
    # API 설정
    api_host = 'https://api.stability.ai'
    api_key = os.getenv("STABILITY_API_KEY")
    engine_id = "stable-diffusion-xl-1024-v1-0"

    if not api_key:
        raise ValueError("STABILITY_API_KEY가 .env 파일에 설정되지 않았습니다.")
        
    print("   - 이미지 생성 요청 중...")

    response = None
    # 이 로직에서는 Text-to-Image만 사용합니다.
    print("   - 모드: Text-to-Image")
    endpoint = f"/v1/generation/{engine_id}/text-to-image"
    
    response = requests.post(
        f"{api_host}{endpoint}",
        headers={"Content-Type": "application/json", "Accept": "application/json", "Authorization": f"Bearer {api_key}"},
        json={
            "text_prompts": [{"text": final_prompt}], "cfg_scale": 7, "height": 1024,
            "width": 1024, "samples": 1, "steps": 30, "style_preset": style_preset
        },
    )

    if response.status_code != 200:
        raise Exception(f"Stability AI API 오류: {response.status_code} - {response.text}")

    data = response.json()
    
    # 결과 이미지 처리 및 저장
    for i, image in enumerate(data["artifacts"]):
        image_bytes = base64.b64decode(image["base64"])
        img = Image.open(io.BytesIO(image_bytes))
        
        print(f"\n 생성된 이미지를 512x512로 리사이즈 중...")
        img_resized = img.resize((512, 512), Image.Resampling.LANCZOS)

        # 전달받은 파일명으로 저장
        img_resized.save(output_filename, "jpeg")
        
        print(f"✅ 이미지가 '{output_filename}' 경로에 저장되었습니다.")
    
    return output_filename


def process_csv_file(csv_path: str):
    """
    CSV 파일을 순회하며 이미지 생성 및 URL 저장을 수행합니다.
    """
    # 1. CSV 파일 불러오기
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        raise Exception(f"오류: '{csv_path}' 파일을 찾을 수 없습니다.")

    # 필수 컬럼 확인
    if '이름' not in df.columns or 'url' not in df.columns:
        raise Exception("오류: CSV 파일에 '이름'과 'url' 열이 모두 필요합니다.")

    # 이미지 저장을 위한 폴더 생성
    output_image_dir = "generated_images_select"
    os.makedirs(output_image_dir, exist_ok=True)
    
    # 결과 CSV 파일 경로 설정
    output_csv_path = os.path.splitext(csv_path)[0] + "_processed.csv"

    total_rows = len(df)
    for index, row in df.iterrows():
        korean_prompt = row['이름']
        
        print(f"\n--- [ {index + 1} / {total_rows} ] 행 처리 중 ---")
        
        # '이름' 열이 비어있으면 건너뛰기
        if pd.isna(korean_prompt) or not str(korean_prompt).strip():
            print("   - '이름' 열이 비어있어 건너뜁니다.")
            continue
            
        # 'url' 열에 이미 값이 있으면 건너뛰기
        if pd.notna(row['url']) and str(row['url']).strip():
            print(f"   - '{korean_prompt}' 항목은 이미 URL({row['url']})이 있어 건너뜁니다.")
            continue

        try:
            # 2. 한글 -> 영어 번역
            print(f"🔄 '{korean_prompt}' 번역 중...")
            english_prompt = translate_with_gpt(korean_prompt)
            print(f"   - 번역 결과: {english_prompt}")

            # 3. 이미지 생성 및 저장
            # 고유한 파일명 생성 (예: generated_images/image_0.jpeg)
            output_filename = os.path.join(output_image_dir, f"image_{index}.jpeg")
            generated_path = generate_final_image(english_prompt, output_filename=output_filename)
            
            # 4. 생성된 이미지 경로를 'url' 열에 저장
            df.loc[index, 'url'] = generated_path
            
            # 5. 5개마다 중간 저장 (스크립트 중단 시 데이터 보존)
            if (index + 1) % 5 == 0:
                df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
                print(f"\n--- 중간 저장 완료 ({output_csv_path}) ---")

        except Exception as e:
            print(f"!! 오류 발생 (행 {index + 1}): {korean_prompt} | {e}")
            print("   - 다음 행으로 넘어갑니다.")
            continue # 오류 발생 시 다음 행으로 계속 진행

    # 6. 모든 작업 완료 후 최종 CSV 파일 저장
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    print(f"\n🎉 모든 작업이 완료되었습니다. 최종 결과가 '{output_csv_path}'에 저장되었습니다.")


if __name__ == "__main__":
    try:
        csv_file_path = input("📂 처리할 CSV 파일의 경로를 입력하세요: ")
        process_csv_file(csv_file_path)

    except Exception as e:
        print(f"\n--- 스크립트 실행 중 심각한 오류 발생 ---")
        print(e)
        print("-----------------------------------")


--- [ 1 / 27 ] 행 처리 중 ---
🔄 '가족/친구와 거실에서 웃으며 대화하는 장면' 번역 중...
   - 번역 결과: A scene of family and friends laughing and conversing joyfully in a warmly lit living room.
📸 'photographic' 스타일로 이미지를 생성합니다.
   - 최종 프롬프트: An ultra-realistic RAW photo of A scene of family and friends laughing and conversing joyfully in a warmly lit living room.. Shot on a Sony α7 IV with an 85mm f/1.4 lens, 8K, cinematic lighting, soft shadows.
   - 스타일 프리셋: photographic
   - 이미지 생성 요청 중...
   - 모드: Text-to-Image

 생성된 이미지를 512x512로 리사이즈 중...
✅ 이미지가 'generated_images_select/image_0.jpeg' 경로에 저장되었습니다.

--- [ 2 / 27 ] 행 처리 중 ---
🔄 '침구 위에서 노트북으로 작업하는 모습' 번역 중...


/var/folders/89/_1v97f8958xcfmysp3lmrmj80000gn/T/ipykernel_12320/2245307052.py:171: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'generated_images_select/image_0.jpeg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'url'] = generated_path


   - 번역 결과: A person working on a laptop while sitting on a bed, surrounded by soft pillows and blankets in a warmly lit bedroom.
📸 'photographic' 스타일로 이미지를 생성합니다.
   - 최종 프롬프트: An ultra-realistic RAW photo of A person working on a laptop while sitting on a bed, surrounded by soft pillows and blankets in a warmly lit bedroom.. Shot on a Sony α7 IV with an 85mm f/1.4 lens, 8K, cinematic lighting, soft shadows.
   - 스타일 프리셋: photographic
   - 이미지 생성 요청 중...
   - 모드: Text-to-Image

 생성된 이미지를 512x512로 리사이즈 중...
✅ 이미지가 'generated_images_select/image_1.jpeg' 경로에 저장되었습니다.

--- [ 3 / 27 ] 행 처리 중 ---
🔄 '옷을 개어 정리하는 모습' 번역 중...
   - 번역 결과: A person neatly folding and organizing clothes in a well-lit, tidy room.
📸 'photographic' 스타일로 이미지를 생성합니다.
   - 최종 프롬프트: An ultra-realistic RAW photo of A person neatly folding and organizing clothes in a well-lit, tidy room.. Shot on a Sony α7 IV with an 85mm f/1.4 lens, 8K, cinematic lighting, soft shadows.
   - 스타일 프리셋: photographic
   - 이미지 생성 요청 중...
   - 

### 이미지 키워드 만들기

In [41]:
import os
import base64
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from PIL import Image
import io

# .env 파일에서 API 키 불러오기
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

if not client.api_key:
    raise ValueError("OpenAI API 키가 .env 파일에 설정되지 않았습니다. 파일을 확인해주세요.")

def encode_image_to_base64(image_path):
    """이미지 파일을 Base64로 인코딩합니다."""
    try:
        with Image.open(image_path) as img:
            # RGBA -> RGB 변환 (PNG 투명도 처리)
            if img.mode == 'RGBA':
                img = img.convert('RGB')
            
            # 이미지 리사이즈 (API 비용 및 속도 최적화)
            img.thumbnail((1024, 1024))
            
            buffered = io.BytesIO()
            img.save(buffered, format="JPEG")
            return base64.b64encode(buffered.getvalue()).decode('utf-8')
    except Exception as e:
        print(f"  - 이미지 처리 오류 ({image_path}): {e}")
        return None

def get_keywords_for_image(base64_image):
    """OpenAI API를 사용하여 이미지로부터 키워드를 추출합니다."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert image analyst. Your task is to extract relevant keywords from an image for a RAG (Retrieval-Augmented Generation) system. Analyze the image and provide a comma-separated list of keywords covering objects, actions, setting, mood, and composition. Respond only with the keywords."
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Extract keywords from this image."},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=200
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"  - OpenAI API 호출 오류: {e}")
        return "키워드 추출 실패"

def process_images_in_folder(folder_path):
    """폴더 내의 모든 이미지를 처리하여 키워드를 추출하고 CSV로 저장합니다."""
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.webp'))]
    
    # ✨ 이 부분을 추가하여 파일 목록을 이름순으로 정렬합니다.
    image_files.sort()
    
    if not image_files:
        print(f"'{folder_path}' 폴더에 처리할 이미지가 없습니다.")
        return

    results = []
    total_images = len(image_files)

    for i, filename in enumerate(image_files):
        print(f"--- [ {i+1} / {total_images} ] 처리 중: {filename} ---")
        image_path = os.path.join(folder_path, filename)
        
        # 1. 이미지 인코딩
        base64_image = encode_image_to_base64(image_path)
        if not base64_image:
            continue

        # 2. 키워드 추출
        keywords = get_keywords_for_image(base64_image)
        print(f"  - 추출된 키워드: {keywords}")
        
        # 3. 결과 저장
        results.append({
            '이미지 경로': f"generated_images/{filename}",
            '이미지 파일명': filename,
            '추출된 키워드': keywords
        })

    # 4. 결과를 DataFrame으로 변환하고 CSV 파일로 저장
    df = pd.DataFrame(results)
    output_path = "image_keywords.csv"
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"\n🎉 모든 작업이 완료되었습니다! 결과가 '{output_path}' 파일에 저장되었습니다.")

if __name__ == "__main__":
    # 이미지가 저장된 폴더 경로를 입력받습니다.
    image_folder = "generated_images" 
    
    if os.path.isdir(image_folder):
        process_images_in_folder(image_folder)
    else:
        print("오류: 유효한 폴더 경로가 아닙니다. 경로를 다시 확인해주세요.")

--- [ 1 / 106 ] 처리 중: image_00001.jpeg ---
  - 추출된 키워드: woman, reading, book, sofa, living room, cozy, plants, bookshelf, framed pictures, natural light, relaxed, home interior
--- [ 2 / 106 ] 처리 중: image_00002.jpeg ---
  - 추출된 키워드: woman, holding mug, sitting, window, sunlight, indoor plants, warm light, cozy atmosphere, sweater, morning, peaceful, relaxed mood, soft focus
--- [ 3 / 106 ] 처리 중: image_00003.jpeg ---
  - 추출된 키워드: people, laughing, sitting, living room, couch, casual clothing, bright, daytime, window, plants, lamps, happiness, socializing, relaxation, gathering
--- [ 4 / 106 ] 처리 중: image_00004.jpeg ---
  - 추출된 키워드: friends, laughing, group, cozy, living room, evening, relaxed, conversation, casual clothing, indoors, drinks, shelf, lamp, plants, happiness, socializing
--- [ 5 / 106 ] 처리 중: image_00005.jpeg ---
  - 추출된 키워드: cozy, relaxed, evening, warm lighting, blanket, woman, living room, pendant lights, cup, dim light, introspection, calm, comfort, peaceful atmosphere


### 로컬 rag 테스트

하이브리드 유사 추출(비슷한게 추출되긴한데 좀 더 성능을 손봐야함)

In [35]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
from PIL import Image
import os
import base64
import openai
import shutil
from dotenv import load_dotenv

# --- 함수 정의 부분 ---

def generate_caption_for_image(client, image_path: str) -> str:
    """GPT-4o Vision 모델을 사용하여 이미지에 대한 상세한 설명을 생성합니다."""
    print(f"   - 🖼️ '{image_path}' 캡션 생성 중...")
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": [
                    {"type": "text", "text": "Describe this image in detail for an image search system. Focus on objects, setting, mood, and actions."},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]}
            ], max_tokens=100
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"    - 캡션 생성 오류: {e}")
        return ""

def build_rag_index(model, client_openai, client_chroma, csv_path: str, image_directory: str):
    """(하이브리드용) 이미지 벡터와 텍스트 벡터를 각각의 컬렉션에 저장합니다."""
    DB_PATH = "./chroma_db"
    if os.path.exists(DB_PATH):
        shutil.rmtree(DB_PATH)
    
    collection_image = client_chroma.get_or_create_collection(name="image_vectors", metadata={"hnsw:space": "cosine"})
    collection_text = client_chroma.get_or_create_collection(name="text_vectors", metadata={"hnsw:space": "cosine"})
    
    df = pd.read_csv(csv_path)
    for index, row in df.iterrows():
        item_id = f"item_{index}"
        image_name = row['이미지 파일명']
        image_path = os.path.join(image_directory, image_name)
        if not os.path.exists(image_path): continue
            
        image_embedding = model.encode(Image.open(image_path))
        caption = generate_caption_for_image(client_openai, image_path)
        tags = str(row['태그'])
        text_to_embed = f"Tags: {tags}. Caption: {caption}"
        text_embedding = model.encode(text_to_embed)
        metadata = {"이름": str(image_name), "태그": tags, "경로": image_path, "캡션": caption}
        
        collection_image.add(embeddings=[image_embedding.tolist()], metadatas=[metadata], ids=[item_id])
        collection_text.add(embeddings=[text_embedding.tolist()], metadatas=[metadata], ids=[item_id])
        print(f"   - ✅ '{item_id}' ({image_name}) 인덱싱 완료")
    print("\n--- ✨ 인덱싱 작업 완료 ---")

def correct_query_spelling(client, query_text: str) -> str:
    """GPT를 이용해 검색어의 오타를 교정합니다."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a spell checker. Correct any spelling mistakes in the user's search query. Respond with only the corrected query."},
                {"role": "user", "content": "a man blowing brithday candles"},
                {"role": "assistant", "content": "a man blowing birthday candles"},
                {"role": "user", "content": query_text}
            ],
            temperature=0, max_tokens=50
        )
        corrected_query = response.choices[0].message.content.strip()
        if corrected_query and corrected_query.lower() != query_text.lower():
            print(f"   - 쿼리 교정: '{query_text}' -> '{corrected_query}'")
            return corrected_query
    except Exception as e:
        print(f"    - 쿼리 교정 오류: {e}")
    return query_text

def final_search(model, client_openai, client_chroma, query_text: str, top_k: int = 5):
    """쿼리 교정 및 강력한 키워드 부스팅을 적용한 최종 RRF 검색 함수"""
    
    # 1. 쿼리 철자 교정
    corrected_query = correct_query_spelling(client_openai, query_text)
    query_words = set(word for word in corrected_query.lower().split() if len(word) > 2)

    print(f"\n--- 💡 스마트 검색 시작: '{corrected_query}' ---")
    
    collection_image = client_chroma.get_collection(name="image_vectors")
    collection_text = client_chroma.get_collection(name="text_vectors")
    
    query_embedding = model.encode(corrected_query).tolist()
    CANDIDATE_COUNT = 20
    image_results = collection_image.query(query_embeddings=[query_embedding], n_results=CANDIDATE_COUNT)
    text_results = collection_text.query(query_embeddings=[query_embedding], n_results=CANDIDATE_COUNT)

    # 2. RRF 점수 계산
    rrf_scores = {}
    k = 60
    for rank, item_id in enumerate(image_results['ids'][0]):
        if item_id not in rrf_scores: rrf_scores[item_id] = {'score': 0, 'metadata': image_results['metadatas'][0][rank]}
        rrf_scores[item_id]['score'] += 1 / (k + rank)

    for rank, item_id in enumerate(text_results['ids'][0]):
        if item_id not in rrf_scores: rrf_scores[item_id] = {'score': 0, 'metadata': text_results['metadatas'][0][rank]}
        rrf_scores[item_id]['score'] += 1 / (k + rank)
        
    # 3. 키워드 부스팅
    KEYWORD_BOOST = 1.0 
    boosted_ids = []
    for item_id, data in rrf_scores.items():
        tags = data['metadata'].get('태그', '').lower()
        if any(word in tags for word in query_words):
             rrf_scores[item_id]['score'] += KEYWORD_BOOST
             boosted_ids.append(item_id)
    
    if boosted_ids:
        print(f"   - 🚀 키워드 부스트 적용: {', '.join(boosted_ids)}")

    # 4. 최종 점수로 정렬 및 결과 출력
    sorted_results = sorted(rrf_scores.items(), key=lambda item: item[1]['score'], reverse=True)
    print("\n--- 🏆 최종 검색 결과 ---")
    for i, (item_id, data) in enumerate(sorted_results[:top_k]):
        boost_info = "(⭐ 키워드 부스트됨)" if data['score'] >= KEYWORD_BOOST else ""
        print(f"  [결과 {i+1}] (최종 점수: {data['score']:.6f}) {boost_info}")
        print(f"    - ID: {item_id}")
        print(f"    - 이름: {data['metadata'].get('이름')}")
        print(f"    - 경로: {data['metadata'].get('경로')}")
        print(f"    - 태그: {data['metadata'].get('태그')}")
        print(f"    - 캡션: {data['metadata'].get('캡션')}")
        print("-" * 20)

# --- 메인 실행 블록 ---
if __name__ == "__main__":
    load_dotenv()
    print("🤖 모델 및 클라이언트를 초기화합니다...")
    model = SentenceTransformer('clip-ViT-B-32')
    client_openai = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    client_chroma = chromadb.PersistentClient(path="./chroma_db")
    print("✅ 초기화 완료.")

    # --- RAG 인덱스 구축 (DB가 없을 때, 최초 한 번만 실행) ---
    # print("--- RAG 인덱스 구축 시작 ---")
    # CSV_FILE_PATH = "여기에 CSV 파일 경로를 입력하세요" 
    # IMAGE_DIRECTORY = "여기에 이미지 폴더 경로를 입력하세요"
    # build_rag_index(model, client_openai, client_chroma, CSV_FILE_PATH, IMAGE_DIRECTORY)
    # print("--- RAG 인덱스 구축 완료 ---\n")
    
    # --- 대화형 지능 검색 루프 ---
    while True:
        user_query = input("\n🖼️ 검색할 내용을 입력하세요 (종료하려면 'exit' 입력): ")
        if user_query.lower() == 'exit':
            print("👋 프로그램을 종료합니다."); break
        if not user_query.strip():
            print("   - 검색어를 입력해주세요."); continue
        
        final_search(model, client_openai, client_chroma, user_query)

🤖 모델 및 클라이언트를 초기화합니다...
✅ 초기화 완료.

--- 💡 스마트 검색 시작: 'smartphone' ---
   - 🚀 키워드 부스트 적용: item_52, item_6

--- 🏆 최종 검색 결과 ---
  [결과 1] (최종 점수: 1.033333) (⭐ 키워드 부스트됨)
    - ID: item_52
    - 이름: image_00053.jpeg
    - 경로: /Users/seoyeong/product-details-service/test/generated_images/image_00053.jpeg
    - 태그: smartphone, hands, screen, photography, editing, outdoors, cityscape, blurred background, touch screen, settings, technology
    - 캡션: The image shows a person holding a smartphone, using a camera or photography app. The phone screen displays a blurry photograph with what seems to be a scenic or urban background, possibly showing a skyline or buildings, although details are not clear. The app interface includes various controls and settings for photography, such as exposure, filters, and additional options for adjusting the image. The person is using their other hand to interact with the touchscreen, perhaps adjusting settings or taking a photo. The background around the person is di

### AZURE 업로드

In [43]:
import os
import pandas as pd
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient

def upload_and_update_csv(local_csv_path: str, local_path_column: str):
    """
    로컬 이미지를 Azure에 업로드하고, 클라우드 URL이 포함된 새 CSV를 생성합니다.
    
    :param local_csv_path: 원본 CSV 파일 경로
    :param local_path_column: 로컬 이미지 경로가 포함된 열의 이름
    """
    # 1. 환경 변수 로드 및 Azure 클라이언트 초기화
    load_dotenv()
    connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
    container_name = os.getenv('AZURE_CONTAINER_NAME')

    if not connect_str or not container_name:
        raise ValueError("Azure 접속 정보를 .env 파일에 설정해주세요.")

    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    
    # 2. 원본 CSV 파일 읽기
    try:
        df = pd.read_csv(local_csv_path)
    except FileNotFoundError:
        print(f"오류: '{local_csv_path}' 파일을 찾을 수 없습니다.")
        return

    # 3. 새로운 'cloud_url' 컬럼 추가
    df['cloud_url'] = ''
    
    print("--- 🔄 이미지 업로드 및 CSV 업데이트 시작 ---")
    
    # 4. CSV의 각 행을 순회하며 작업 수행
    for index, row in df.iterrows():
        local_path = row[local_path_column]
        if not isinstance(local_path, str) or not os.path.exists(local_path):
            print(f"  - 경고: '{local_path}' 경로가 유효하지 않아 건너뜁니다.")
            continue

        file_name = os.path.basename(local_path)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
        
        print(f"  - 업로드 중: {local_path} -> Azure ({file_name})")
        
        # 5. 로컬 파일 업로드
        with open(local_path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)
            
        # 6. 업로드된 파일의 URL을 DataFrame에 저장
        df.loc[index, 'cloud_url'] = blob_client.url

    # 7. 모든 정보가 업데이트된 새 CSV 파일 저장
    output_csv_path = os.path.splitext(local_csv_path)[0] + "_cloud.csv"
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    
    print(f"\n--- 🎉 작업 완료! '{output_csv_path}' 파일이 생성되었습니다. ---")


if __name__ == "__main__":
    # ------------------ 설정 ------------------
    # 1. 실제 사용하는 CSV 파일명으로 변경하세요.
    SOURCE_CSV_FILE = 'image_keywords.csv' 
    # 2. CSV 파일 내에서 로컬 경로가 담긴 열의 이름을 정확히 입력하세요.
    LOCAL_PATH_COLUMN_NAME = '이미지 경로' 
    # ----------------------------------------
    
    upload_and_update_csv(
        local_csv_path=SOURCE_CSV_FILE, 
        local_path_column=LOCAL_PATH_COLUMN_NAME
    )

--- 🔄 이미지 업로드 및 CSV 업데이트 시작 ---
  - 업로드 중: generated_images/image_00001.jpeg -> Azure (image_00001.jpeg)
  - 업로드 중: generated_images/image_00002.jpeg -> Azure (image_00002.jpeg)
  - 업로드 중: generated_images/image_00003.jpeg -> Azure (image_00003.jpeg)
  - 업로드 중: generated_images/image_00004.jpeg -> Azure (image_00004.jpeg)
  - 업로드 중: generated_images/image_00005.jpeg -> Azure (image_00005.jpeg)
  - 업로드 중: generated_images/image_00006.jpeg -> Azure (image_00006.jpeg)
  - 업로드 중: generated_images/image_00007.jpeg -> Azure (image_00007.jpeg)
  - 업로드 중: generated_images/image_00008.jpeg -> Azure (image_00008.jpeg)
  - 업로드 중: generated_images/image_00009.jpeg -> Azure (image_00009.jpeg)
  - 업로드 중: generated_images/image_00010.jpeg -> Azure (image_00010.jpeg)
  - 업로드 중: generated_images/image_00011.jpeg -> Azure (image_00011.jpeg)
  - 업로드 중: generated_images/image_00012.jpeg -> Azure (image_00012.jpeg)
  - 업로드 중: generated_images/image_00013.jpeg -> Azure (image_00013.jpeg)
  - 업로드 중: generate

azure 업로드 테스트

In [29]:
import os
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient

def test_single_upload():
    """단일 파일을 Azure Blob Storage에 업로드하여 연결을 테스트합니다."""
    
    # --- 설정 (이 부분을 자신의 환경에 맞게 수정하세요) ---
    # 1. 실제 존재하는 이미지 파일의 전체 경로를 입력하세요.
    LOCAL_FILE_TO_UPLOAD = "/Users/seoyeong/product-details-service/test/generated_images/image_00001.jpeg"
    # 2. Azure에 저장될 파일명을 지정합니다. (보통 원본과 동일하게)
    BLOB_NAME_TO_SAVE = "test-image.jpeg"
    # ---------------------------------------------------

    print("--- Azure Blob Storage 연결 테스트 시작 ---")
    
    try:
        # 1. .env 파일 로드 및 연결 정보 확인
        load_dotenv()
        connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
        container_name = os.getenv('AZURE_CONTAINER_NAME')

        if not connect_str or not container_name:
            raise ValueError("AZURE_STORAGE_CONNECTION_STRING 또는 AZURE_CONTAINER_NAME이 .env 파일에 없습니다.")
        
        if not os.path.exists(LOCAL_FILE_TO_UPLOAD):
            raise FileNotFoundError(f"테스트할 로컬 파일을 찾을 수 없습니다: {LOCAL_FILE_TO_UPLOAD}")

        print(f"컨테이너: '{container_name}'에 연결 시도 중...")
        
        # 2. Blob Service 클라이언트 생성
        blob_service_client = BlobServiceClient.from_connection_string(connect_str)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=BLOB_NAME_TO_SAVE)

        # 3. 파일 업로드
        print(f"파일 업로드 중: '{LOCAL_FILE_TO_UPLOAD}'")
        with open(LOCAL_FILE_TO_UPLOAD, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)
            
        print("\n--- ✅ 업로드 성공! ---")
        print("생성된 URL:", blob_client.url)
        print("Azure Portal에서 컨테이너를 확인하여 'test-image.jpeg' 파일이 있는지 확인해보세요.")

    except Exception as e:
        print("\n--- ❌ 오류 발생! ---")
        print("오류 메시지:", e)

if __name__ == "__main__":
    test_single_upload()

--- Azure Blob Storage 연결 테스트 시작 ---
컨테이너: 'images'에 연결 시도 중...
파일 업로드 중: '/Users/seoyeong/product-details-service/test/generated_images/image_00001.jpeg'

--- ✅ 업로드 성공! ---
생성된 URL: https://stgsangsang.blob.core.windows.net/images/test-image.jpeg
Azure Portal에서 컨테이너를 확인하여 'test-image.jpeg' 파일이 있는지 확인해보세요.


### 확인용 검색(ID 또는 의미기반 검색 둘다 가능)

In [3]:
import chromadb
from sentence_transformers import SentenceTransformer
import os

# --- 1. 모델 및 DB 클라이언트 초기화 ---

print("🤖 CLIP 모델을 로드합니다...")
model = SentenceTransformer('clip-ViT-B-32')

DB_PATH = "./chroma_db"
if not os.path.exists(DB_PATH):
    raise FileNotFoundError(f"'{DB_PATH}' 폴더를 찾을 수 없습니다. 먼저 인덱싱을 실행해주세요.")

client_chroma = chromadb.PersistentClient(path=DB_PATH)

collection_name = "image_rag_collection"
try:
    collection = client_chroma.get_collection(name=collection_name)
    print(f"✅ '{collection_name}' 컬렉션을 성공적으로 불러왔습니다.")
except Exception as e:
    raise ValueError(f"'{collection_name}' 컬렉션을 불러오는 데 실패했습니다: {e}")


# --- 2. 기능별 함수 정의 ---

def search_images_by_meaning(query_text: str, top_k: int = 3):
    """의미 기반으로 가장 유사한 이미지를 검색합니다."""
    print(f"\n--- 🔍 의미 기반 검색 시작: '{query_text}' ---")
    
    query_embedding = model.encode(query_text).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    
    # (결과 출력 부분은 이전과 동일)
    if not results['ids'][0]: print("   - 검색 결과가 없습니다."); return

    print("\n--- 🏆 검색 결과 ---")
    for i, item_id in enumerate(results['ids'][0]):
        metadata = results['metadatas'][0][i]
        distance = results['distances'][0][i]
        print(f"  [결과 {i+1}] (유사도 점수: {1-distance:.4f})")
        print(f"    - ID: {item_id}")
        print(f"    - 이름: {metadata.get('이름')}")
        print(f"    - 캡션: {metadata.get('캡션')}")
        print("-" * 20)

def get_item_by_id(item_id: str):
    """ID로 특정 아이템의 정보를 직접 조회합니다."""
    print(f"\n--- 🆔 ID로 직접 조회 시작: '{item_id}' ---")

    result = collection.get(ids=[item_id])

    if not result['ids']:
        print(f"   - '{item_id}'에 해당하는 아이템을 찾을 수 없습니다."); return

    metadata = result['metadatas'][0]
    
    print("--- 📄 조회된 메타데이터 ---")
    print(f"  - 이름: {metadata.get('이름')}")
    print(f"  - 태그: {metadata.get('태그')}")
    print(f"  - 캡션: {metadata.get('캡션')}")
    print(f"  - 경로: {metadata.get('경로')}")
    print("-" * 20)

# --- 3. 메인 실행 블록 ---

if __name__ == "__main__":
    while True:
        user_query = input("\n🖼️ 검색어 또는 ID('item_xx')를 입력하세요 (종료: 'exit'): ")
        
        if user_query.lower() == 'exit':
            print("👋 프로그램을 종료합니다."); break
        if not user_query.strip():
            print("   - 검색어나 ID를 입력해주세요."); continue
        
        # ▼▼▼ 입력 값에 따라 다른 함수를 호출하는 로직 ▼▼▼
        if user_query.lower().startswith("item_"):
            get_item_by_id(user_query) # ID 조회
        else:
            search_images_by_meaning(user_query) # 의미 기반 검색

🤖 CLIP 모델을 로드합니다...
✅ 'image_rag_collection' 컬렉션을 성공적으로 불러왔습니다.

--- 🆔 ID로 직접 조회 시작: 'item_96' ---
--- 📄 조회된 메타데이터 ---
  - 이름: image_00097.jpeg
  - 태그: shower, bathroom, steam, towel, tiles, wet hair, back view, light, hygiene, relaxation
  - 캡션: The image depicts a person in a bathroom setting. They are standing with their back to the camera and have wet hair, suggesting they have just finished showering. The atmosphere is steamy, indicating recent use of hot water. The person is wrapped in a white towel around their waist.

The bathroom features a tiled interior with light-colored tiles covering the walls. There is a showerhead visible on the left side of the image, and a mirror along with some toiletries can be seen on a shelf or led
  - 경로: /Users/seoyeong/product-details-service/test/generated_images/image_00097.jpeg
--------------------
   - 검색어나 ID를 입력해주세요.
   - 검색어나 ID를 입력해주세요.
   - 검색어나 ID를 입력해주세요.
   - 검색어나 ID를 입력해주세요.
   - 검색어나 ID를 입력해주세요.
   - 검색어나 ID를 입력해주세요.
   - 검색어나 ID를